## Fully automated degradation process

1. Generate degraded MOF structure (select degradation percent)
2. Solvate degraded MOF with water and drug molecules (and maybe deleted linkers)
3. Assign force field parameters to solvated structure
4. Generate LAMMPS files
5. Run simulation

## Questions

- Is Node + Linker degradation fine?
- Which linkers/nodes to delete for degradation (start from outer shell??)
- Should we add the degraded linkers/nodes in solution??

In [ ]:
import os
from thermof.trajectory.tools import center_of_mass
from file_io import write_xyz
from degradation_tools import pack_mof, degrade_mof
from irmof1 import irmof1
from packmol import Packmol

In [ ]:
W_WATER = 0.1       # Water weight fraction
W_PZA = 0.1         # PZA weight fraction
W_NIZ = 0.0         # NIZ weight fraction
W_RFP = 0.0         # RFP weight fraction
D_MOF = (0.8, 1.0)  # Degradation fractions (linkers, nodes)
P_MOF = [2, 2, 2]   # MOF packing coefficient

## Generating MOF

In [ ]:
mof = irmof1.copy()
packed_mof = pack_mof(mof['atoms'], mof['coordinates'], mof['cell'], P_MOF)
dmof_file = 'IRMOF1-%s-L%i-N%i.xyz' % (''.join([str(i) for i in P_MOF]), D_MOF[0] * 100, D_MOF[1] * 100)
degrade_mof(packed_mof, D_MOF, os.path.join('packmol', dmof_file))

## Solvating with Packmol

In [ ]:
edge_tol = 2                  # Empty space on the edges
solv_shell = [15, 15, 15]     # Solvation shell length
box_size = [int(packed_mof['cell'][i]) + 2 * solv_shell[i] for i in range(3)] 
pmol = Packmol()
dmof_position = {'fixed': '%.1f %.1f %.1f 0. 0. 0.' % (box_size[0] / 2, box_size[1] / 2, box_size[2] / 2), 'centerofmass': ''}
pmol.add_structure({'structure': dmof_file, 'number': '1', 'position': dmof_position})

### WATER ----------------------------
n_water = 100
water_position = {'inside box': '%.1f %.1f %.1f %.1f %.1f %.1f' % (edge_tol, edge_tol, edge_tol, box_size[0] - edge_tol, box_size[1] - edge_tol, box_size[2] - edge_tol)}
pmol.add_structure({'structure': 'water.xyz', 'number': n_water, 'position': water_position})

### PZA -------------------------------
n_pza = 20
pza_position = {'inside box': '%.1f %.1f %.1f %.1f %.1f %.1f' % (solv_shell[0], solv_shell[1], solv_shell[2], box_size[0] - solv_shell[0], box_size[1] - solv_shell[1], box_size[2] - solv_shell[2])}
pmol.add_structure({'structure': 'pza.xyz', 'number': n_pza, 'position': pza_position})

### LINKER ----------------------------

### NODE ------------------------------

run_dir = 'pmol-temp'
source_dir = 'packmol'

In [ ]:
pmol.run(run_dir, source_dir)

## Assign force field parameters

In [ ]:
# Run lammps_interface

## LAMMPS

In [ ]:
# Modify input files
# Create job submission file

In [ ]:
linker_atoms = [irmof1['atoms'][i] for i in irmof1['linkers'][0]]
linker_coors = [irmof1['coordinates'][i] for i in irmof1['linkers'][0]]

node_atoms = [irmof1['atoms'][i] for i in irmof1['nodes'][0]]
node_coors = [irmof1['coordinates'][i] for i in irmof1['nodes'][0]]
write_xyz('node.xyz', node_atoms, node_coors)
write_xyz('linker.xyz', linker_atoms, linker_coors)